In [24]:
with open("test_input_1.txt", "r") as f:
    test_input_1 = f.read()
with open("test_input_2.txt", "r") as f:
    test_input_2 = f.read()

test_input_1


'RL\n\nAAA = (BBB, CCC)\nBBB = (DDD, EEE)\nCCC = (ZZZ, GGG)\nDDD = (DDD, DDD)\nEEE = (EEE, EEE)\nGGG = (GGG, GGG)\nZZZ = (ZZZ, ZZZ)'

In [25]:
def get_steps(raw_input):
    return raw_input.split("\n")[0]

get_steps(test_input_1)

'RL'

In [26]:
def get_route(raw_input):
    return raw_input.split("\n\n")[1].split("\n")

get_route(test_input_1)


['AAA = (BBB, CCC)',
 'BBB = (DDD, EEE)',
 'CCC = (ZZZ, GGG)',
 'DDD = (DDD, DDD)',
 'EEE = (EEE, EEE)',
 'GGG = (GGG, GGG)',
 'ZZZ = (ZZZ, ZZZ)']

In [27]:
from collections import namedtuple
Node = namedtuple("Node", "l r")

In [28]:
def build_node(line):
    node, _, raw_left, raw_right = line.split()
    left = raw_left[1:-1]
    right = raw_right[:-1]
    return {node:Node(left, right)}

build_node(get_route(test_input_1)[0])

{'AAA': Node(l='BBB', r='CCC')}

In [29]:
def build_graph(lines):
    graph = dict()
    for line in lines:
        graph = graph | build_node(line)
    return graph

build_graph(get_route(test_input_1))

{'AAA': Node(l='BBB', r='CCC'),
 'BBB': Node(l='DDD', r='EEE'),
 'CCC': Node(l='ZZZ', r='GGG'),
 'DDD': Node(l='DDD', r='DDD'),
 'EEE': Node(l='EEE', r='EEE'),
 'GGG': Node(l='GGG', r='GGG'),
 'ZZZ': Node(l='ZZZ', r='ZZZ')}

In [30]:
from itertools import cycle
START_NODE = "AAA"
END_NODE = "ZZZ"
def calculate_steps_to_end(graph, steps):
    current_node = graph[START_NODE]
    for i, step in enumerate(cycle(steps)):
        new_node_id = current_node.l if step == "L" else current_node.r

        if new_node_id == END_NODE:
            return i+1
        
        current_node = graph[new_node_id]

calculate_steps_to_end(build_graph(get_route(test_input_1)), get_steps(test_input_1))

2

In [31]:
def get_result_1(raw_input):
    return calculate_steps_to_end(build_graph(get_route(raw_input)), get_steps(raw_input))

get_result_1(test_input_2)

6